# Managing Analyzers in Your Resource

This notebook demonstrates how to create a simple analyzer and manage its lifecycle.

## Prerequisites
1. Ensure your Azure AI service is configured following the [configuration steps](../README.md#configure-azure-ai-service-resource).
2. Install the required packages to run this sample.

In [ ]:
%pip install -r ../requirements.txt

## Create the Azure AI Content Understanding Client

> The [AzureContentUnderstandingClient](../python/content_understanding_client.py) is a utility class that provides functions to interact with the Content Understanding API. Before the official release of the Content Understanding SDK, this client serves as a lightweight SDK.

> Fill the constants **AZURE_AI_ENDPOINT**, **AZURE_AI_API_VERSION**, and **AZURE_AI_API_KEY** with your Azure AI Service details.

> ⚠️ Important:
Update the code below to match your Azure authentication method.
Look for the `# IMPORTANT` comments and modify those sections accordingly.
If you skip this step, the sample might not run correctly.

> ⚠️ Note: Using a subscription key works, but using Azure Active Directory (AAD) token-based authentication is more secure and highly recommended for production environments.

In [ ]:
import logging
import json
import os
import sys
import uuid
from dotenv import load_dotenv
from azure.storage.blob import ContainerSasPermissions
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from azure.ai.contentunderstanding.aio import ContentUnderstandingClient
from azure.ai.contentunderstanding.models import (
    ContentAnalyzer,
    ContentAnalyzerConfig,
    FieldSchema,
    FieldDefinition,
    FieldType,
    GenerationMethod,
    AnalysisMode,
    ProcessingLocation,
)

# Add the parent directory to the Python path to import the sample_helper module
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'python'))
from extension.document_processor import DocumentProcessor
from extension.sample_helper import extract_operation_id_from_poller, PollerType, save_json_to_file

load_dotenv()
logging.basicConfig(level=logging.INFO)

endpoint = os.environ.get("AZURE_CONTENT_UNDERSTANDING_ENDPOINT")
# Return AzureKeyCredential if AZURE_CONTENT_UNDERSTANDING_KEY is set, otherwise DefaultAzureCredential
key = os.getenv("AZURE_CONTENT_UNDERSTANDING_KEY")
credential = AzureKeyCredential(key) if key else DefaultAzureCredential()
# Create the ContentUnderstandingClient
client = ContentUnderstandingClient(endpoint=endpoint, credential=credential)
print("✅ ContentUnderstandingClient created successfully")

try:
    processor = DocumentProcessor(client)
    print("✅ DocumentProcessor created successfully")
except Exception as e:
    print(f"❌ Failed to create DocumentProcessor: {e}")
    raise

## Create a Simple Analyzer
First, we create an analyzer from a template to extract invoice fields.

In [ ]:
analyzer_id = "management-sample-" + str(uuid.uuid4())

# Create a custom analyzer using object model
print(f"🔧 Creating custom analyzer '{analyzer_id}'...")

call_analyzer = ContentAnalyzer(
    base_analyzer_id="prebuilt-callCenter",
    description="Sample call recording analytics",
    config=ContentAnalyzerConfig(
        return_details=True,
        locales=["en-US"]
    ),
    field_schema=FieldSchema(
        fields={
            "Summary": FieldDefinition(
                type=FieldType.STRING,
                method=GenerationMethod.GENERATE,
                description="A one-paragraph summary"
            ),
            "Topics": FieldDefinition(
                type=FieldType.ARRAY,
                method=GenerationMethod.GENERATE,
                description="Top 5 topics mentioned",
                items_property={
                    "type": "string"
                }
            ),
            "Companies": FieldDefinition(
                type=FieldType.ARRAY,
                method=GenerationMethod.GENERATE,
                description="List of companies mentioned",
                items_property={
                    "type": "string"
                }
            ),
            "People": FieldDefinition(
              type=FieldType.ARRAY,
              method=GenerationMethod.GENERATE,
              description="List of people mentioned",
              items_property={
                  "type": FieldType.OBJECT,
                  "properties": {
                      "Name": {
                          "type": FieldType.STRING,
                          "description": "Person's name"
                  },
                  "Role": {
                    "type": FieldType.STRING,
                    "description": "Person's title/role"
                  }
                }
              }
            ),
            "Sentiment": FieldDefinition(
              type=FieldType.STRING,
              method=GenerationMethod.CLASSIFY,
              description="Overall sentiment",
              enum=[
                "Positive",
                "Neutral",
                "Negative"
              ]
            ),
            "Categories": FieldDefinition(
              type=FieldType.ARRAY,
              method=GenerationMethod.CLASSIFY,
              description="List of relevant categories",
              items_property={
                "type": FieldType.STRING,
                "enum": [
                  "Agriculture",
                  "Business",
                  "Finance",
                  "Health",
                  "Insurance",
                  "Mining",
                  "Pharmaceutical",
                  "Retail",
                  "Technology",
                  "Transportation"
                ]
              }
            )
        }
    )
)

# Start the analyzer creation operation
poller = await client.content_analyzers.begin_create_or_replace(
    analyzer_id=analyzer_id,
    resource=call_analyzer,
)

# Extract operation ID from the poller
operation_id = extract_operation_id_from_poller(
    poller, PollerType.ANALYZER_CREATION
)
print(f"📋 Extracted creation operation ID: {operation_id}")

# Wait for the analyzer to be created
print(f"⏳ Waiting for analyzer creation to complete...")
await poller.result()
print(f"✅ Analyzer '{analyzer_id}' created successfully!")

## List All Analyzers in Your Resource

After successfully creating an analyzer, you can use it to analyze our input files. You can also list all analyzers available in your resource.

In [ ]:
response = client.content_analyzers.list()
analyzers = [analyzer async for analyzer in response]

print(f"✅ Found {len(analyzers)} analyzers")

# Display detailed information about each analyzer
for i, analyzer in enumerate(analyzers, 1):
    print(f"🔍 Analyzer {i}:")
    print(f"   ID: {analyzer.analyzer_id}")
    print(f"   Description: {analyzer.description}")
    print(f"   Status: {analyzer.status}")
    print(f"   Created at: {analyzer.created_at}")

    # Check if it's a prebuilt analyzer
    if analyzer.analyzer_id.startswith("prebuilt-"):
        print(f"   Type: Prebuilt analyzer")
    else:
        print(f"   Type: Custom analyzer")

    # Show tags if available
    if hasattr(analyzer, "tags") and analyzer.tags:
        print(f"   Tags: {analyzer.tags}")

## Get Analyzer Details by ID

Keep track of the analyzer ID when you create it. Use the ID to retrieve detailed analyzer definitions later.

In [ ]:
retrieved_analyzer: ContentAnalyzer = await client.content_analyzers.get(analyzer_id=analyzer_id)
print(f"✅ Analyzer '{analyzer_id}' retrieved successfully!")
print(f"   Description: {retrieved_analyzer.description}")
print(f"   Status: {retrieved_analyzer.status}")
print(f"   Created at: {retrieved_analyzer.created_at}")

## Delete an Analyzer
If you no longer need an analyzer, delete it using its ID.

In [ ]:
# Clean up: delete the analyzer (demo purposes only)
# Note: You can leave the analyzer for later use if desired
print(f"🗑️  Deleting analyzer '{analyzer_id}' (demo cleanup)...")
await client.content_analyzers.delete(analyzer_id=analyzer_id)
print(f"✅ Analyzer '{analyzer_id}' deleted successfully!")